In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import math
import os
os.chdir('/content/drive/My Drive')

import numpy as np
data = np.load('kmeans_data.npy')


Mounted at /content/drive


In [ ]:
def length(a,b):
  return abs(a[0]-b[0]) + abs(a[1] - b[1])

def distance(a,b):
  return (a[0]-b[0])**2 + (a[1]-b[1])**2

def k_means(data,k):
  n = len(data)
  centroids = data[np.random.choice(data.shape[0], k, replace=False)]
  centroids_plag = centroids.copy()
  points = [-1 for i in range(n)]

  for l in range(1000):
    for i in range(n):
      min = length(data[i],centroids[k-1])
      points[i] = k - 1
      for j in range(k-1):
          if(length(data[i], centroids[j]) < min):
            min = length(data[i], centroids[j])
            points[i] = j

    for i in range(k):
      count = 0
      x_sum = 0
      y_sum = 0
      for j in range(n):
        if(points[j] == i):
          count+=1
          x_sum += data[j][0]
          y_sum += data[j][1]

      x_sum/= count
      y_sum/= count
      centroids_plag[i] = [x_sum, y_sum]

    for i in range(k):
      if(length(centroids_plag[i], centroids[i]) > (1e-5)):
        break
      if(i==k-1):
        return centroids_plag, points
    
    centroids = centroids_plag.copy()

  return centroids, points


In [ ]:
def silhoutte(data):
  bleh = [-1 for i in range(9)]
  for k in range(2,10):
    centroids, points = k_means(data,k)
    for i in range(len(points)):
      center = centroids[points[i]]
      count = 0
      a = 0
      for j in range(len(points)):
        if(points[j] == points[i] and i!=j):
          a += length(data[i],data[j])
          count+=1
      check_ci = count
      a/=count

      for j in range(len(centroids)):
        b = np.inf
        count = 0
        if(points[i]!=j):
          s = 0
          for l in range(len(points)):
            if(points[l] == j):
              s+=length(data[l], data[i])
              count += 1
          s /= count
          b = min(s,b)

      coefficient = float(b - a) / (max(a,b))
      if(check_ci == 1):
        coefficient = 0
      bleh[k-1] = coefficient

  return bleh


In [ ]:
samples = silhoutte(data)

In [ ]:
k = np.argmax(samples,axis=0)
print(k)
print(samples)

2
[-1, 0.2854313002067744, nan, 0.8769053530653804, 0.8374080589005963, 0.9206707627247879, 0.8329385672114494, nan, 0.9440824263259522]


In [ ]:
def objective_function(data, membership_matrix, fuzzy_centroids, m):
  n = len(data)
  c = len(fuzzy_centroids)
  sum = 0
  for i in range(n):
    count = 0
    for j in range(c):
      count += (membership_matrix[i][j]**m) * (distance(data[i],fuzzy_centroids[j]))
    sum+=count

  return sum

In [ ]:
def funny_c_means(c,m, max_iterations=100):
  n = len(data)
  centroid = data[np.random.choice(data.shape[0], c, replace=False)]
  membership_matrix = np.zeros((n, c))
  for s in range(max_iterations):
    for i in range(n):
      for j in range(c):
        a = length(data[i], centroid[j])
        sum = 0
        for l in range(c):
          b = length(data[i], centroid[l])
          sum += ((a/b)**((2/m) - 1))

        membership_matrix[i][j] = 1/sum
    fuzzy_centroids = np.zeros((c,2))
    for i in range(c):
      num = [0,0]
      den = 0
      for j in range(n):
        num += (data[j] * (membership_matrix[j][i]**m))
        den += (membership_matrix[j][i]**m)
      fuzzy_centroids[i] = num/den
    obj_func_val = objective_function(data, membership_matrix, fuzzy_centroids, m)
    if obj_func_val < 0.3:
      break
  return obj_func_val


In [ ]:
value = np.inf
print(k)
value = funny_c_means(k,2, 100)

print(value)

2


<ipython-input-11-79228c3ffc47>:12: RuntimeWarning: invalid value encountered in double_scalars
  sum += ((a/b)**((2/m) - 1))
<ipython-input-11-79228c3ffc47>:12: RuntimeWarning: divide by zero encountered in double_scalars
  sum += ((a/b)**((2/m) - 1))


46611.74570570585
